In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 62 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 11.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=964baf5c4971e10bd415a1b88b942d5eb83eed9632ee3233891f78d36b31534d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 13.6 MB/s 
     |████████████████████████████████| 769 kB 13.4 MB/s 
     |████████████████████████████████| 3.0 MB 40.1 MB/s 
     |████████████████████████████████| 895 kB 42.3 MB/s 


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-2hid_q0s
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-2hid_q0s
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=53f194e8bd09c0ea2c87e54c4ed7c8ed7a724e61cd61e6464505add6b2b4532b
  Stored in directory: /tmp/pip-ephem-wheel-cache-giw1snmw/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

train = pd.read_csv("/content/drive/My Drive/ratings_train.txt", sep='\t')
test = pd.read_csv("/content/drive/My Drive/ratings_test.txt", sep='\t')

train = train.dropna()
test = test.dropna()

print(train.shape)
print(test.shape)

(149995, 3)
(49997, 3)


In [6]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [7]:
train_data_list = []
for q, label in zip(train['document'], train['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    train_data_list.append(data)

In [8]:
test_data_list = []
for q, label in zip(test['document'], test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    test_data_list.append(data)

### KoBERT 입력 데이터로 변환


In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
      
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)  # 토큰화, 패딩 수행

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
# Setting parameters
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_data_list, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data_list, 0, 1, tok, max_len, True, False)

using cached model


In [12]:
data_train[0]

(array([   2, 3093, 1698, 6456,  517,   54,  517,   54, 4368, 4396, 7316,
        5655, 5703, 2073,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

1. 패딩된 sequence
2. 길이, 타입
3. attention mask sequence (1로 패딩된 값들은 연산을 수행하지 않도록)

In [13]:
# torch 형식
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

### KoBERT 학습 모델

In [14]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        
        # 분류기
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        # 실제 단어 토큰이 있는 위치의 길이 만큼만 1로 설정
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
            
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
            
        return self.classifier(out)

In [15]:
# 모델 불러오기
device = torch.device("cuda:0")
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [16]:
#정확도 측정
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    
    return train_acc

In [17]:
#optimizer, schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

### KoBERT 모델 학습

In [18]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7316799759864807 train acc 0.46875
epoch 1 batch id 201 loss 0.5680432915687561 train acc 0.5313277363184079
epoch 1 batch id 401 loss 0.46930018067359924 train acc 0.6562110349127181
epoch 1 batch id 601 loss 0.3800261616706848 train acc 0.716150166389351
epoch 1 batch id 801 loss 0.38230976462364197 train acc 0.7501755617977528
epoch 1 batch id 1001 loss 0.3691583573818207 train acc 0.7706043956043956
epoch 1 batch id 1201 loss 0.38541948795318604 train acc 0.7849058076602831
epoch 1 batch id 1401 loss 0.48130127787590027 train acc 0.7957374197002142
epoch 1 batch id 1601 loss 0.30197110772132874 train acc 0.804955886945659
epoch 1 batch id 1801 loss 0.26669853925704956 train acc 0.8121616463076069
epoch 1 batch id 2001 loss 0.4319404363632202 train acc 0.8186531734132934
epoch 1 batch id 2201 loss 0.29611220955848694 train acc 0.8236242049068605
epoch 1 train acc 0.8272057182117628


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/782 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8998576750934487


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2673775553703308 train acc 0.90625
epoch 5 batch id 201 loss 0.041477251797914505 train acc 0.9738805970149254
epoch 5 batch id 401 loss 0.017895059660077095 train acc 0.9751013092269327
epoch 5 batch id 601 loss 0.223030686378479 train acc 0.9761595257903494
epoch 5 batch id 801 loss 0.19920606911182404 train acc 0.9763966916354557
epoch 5 batch id 1001 loss 0.025157855823636055 train acc 0.9771010239760239
epoch 5 batch id 1201 loss 0.008830444887280464 train acc 0.9776618442964197
epoch 5 batch id 1401 loss 0.10841246694326401 train acc 0.9779175588865097
epoch 5 batch id 1601 loss 0.013195483013987541 train acc 0.9783533728919426
epoch 5 batch id 1801 loss 0.048511914908885956 train acc 0.9788659078289839
epoch 5 batch id 2001 loss 0.07680823653936386 train acc 0.9790651549225388
epoch 5 batch id 2201 loss 0.024869101122021675 train acc 0.9790507155838255
epoch 5 train acc 0.9792488801194539


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 5 test acc 0.8986388451701751


### 새로운 문장으로 테스트

In [23]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정적")

            elif np.argmax(logits) == 1:
                test_eval.append("긍정적")

        print(">> 해당 리뷰는 " + test_eval[0])

using cached model


In [ ]:
end = 1
while end == 1 :
    sentence = input("리뷰 내용 입력 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

>> 해당 리뷰는 부정적


>> 해당 리뷰는 부정적


>> 해당 리뷰는 부정적


>> 해당 리뷰는 긍정적


>> 해당 리뷰는 부정적


>> 해당 리뷰는 부정적


>> 해당 리뷰는 부정적


>> 해당 리뷰는 긍정적


